In [1]:
import numpy as np
import matplotlib.pyplot as plt
#%config InlineBackend.figure_format = 'retina'  # For sharper figures, but it takes more time
import scipy as sp
from copy import deepcopy 

from lisatools.utils.constants import *
from lisatools.sensitivity  import SensitivityMatrix, AET1SensitivityMatrix, AE1SensitivityMatrix
from lisatools.analysiscontainer import AnalysisContainer
from lisatools.datacontainer import DataResidualArray

from bbhx.waveforms.phenomhm import PhenomHMAmpPhase
from bbhx.waveformbuild import BBHWaveformFD
from bbhx.utils.interpolate import CubicSplineInterpolant

import noise_generation as noise_generation
from tools.LISASimulator import LISASimulator, signal_time_to_freq_domain
from tools.likelihood import get_dh, get_hh

No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy
No CuPy or GPU response available.


Generating Simulated LISA data with colored noise and MBHB signal(s)

In [2]:
Tobs = YRSID_SI
dt = 5.
include_T_channel = False

wave_gen = BBHWaveformFD(amp_phase_kwargs=dict(run_phenomd=False))
sim = LISASimulator(Tobs=Tobs, dt=dt, wave_gen=wave_gen, include_T_channel=include_T_channel)

# TODO: Make it work for multiple MBHBs in the signal
f_ref = 0.0
phi_ref = 0.0
m1 = 1e6
m2 = 5e5
a1 = 0.2
a2 = 0.4
dist = 10e3 * PC_SI * 1e6  # 3e3 in Mpc
inc = np.pi/3
beta = np.pi/4.
lam = np.pi/5.
psi = np.pi/6.
t_ref = 0.5 * YRSID_SI  # in the SSB reference frame

parameters = np.array([m1, m2, a1, a2, dist, phi_ref, f_ref, inc, lam, beta, psi, t_ref])

modes = [(2,2), (2,1), (3,3), (3,2), (4,4), (4,3)]
waveform_kwargs = dict(direct=False, fill=True, squeeze=False, length=1024)

signal_with_noise_t = sim(seed = 42, parameters=parameters, modes=modes, waveform_kwargs=waveform_kwargs)

3 (2, 6311628) (1, 2, 6311628)


In [3]:
snr_from_lisatools = sim.SNR_optimal_lisatools()
snr_my_code = sim.SNR_optimal()
print(snr_from_lisatools, snr_my_code)

[3530.14340371] [3530.14340371]


In [12]:
data = sim.signal_with_noise_f
f_array = np.fft.rfftfreq(sim.signal_with_noise_t.shape[2])  # returns the correct frequency array for the signal with noise
f_array[0] = f_array[1]  # avoid zero frequency
data[0]

array([[ 2.87997753e-15+0.00000000e+00j, -8.01664455e-16-9.14272412e-15j,
         1.87766792e-15+4.67276244e-15j, ...,
        -5.23505584e-17+5.48756912e-17j,  1.68362572e-17+4.01699770e-17j,
        -6.00596015e-17+0.00000000e+00j],
       [ 7.58040788e-15+0.00000000e+00j,  1.06930310e-15+2.58035370e-15j,
        -5.80231866e-15+8.54461968e-16j, ...,
         2.63527057e-17+7.97395671e-17j, -4.20193941e-17+8.92831265e-17j,
         4.73211831e-17+0.00000000e+00j]], shape=(2, 3155815))

In [13]:
#sim.df, len(sim.signal_with_noise_t[0,0])*sim.dt, sim.Tobs, sim.Tobs / sim.dt, sim.time.shape, sim.freq

In [14]:
template = wave_gen(*parameters, freqs=f_array, modes=modes, **waveform_kwargs)[0]
if include_T_channel==False:
    template = template[:2]  # remove T channel if not included

In [15]:
dh = get_dh(data, template, sens_mat=sim.sens_mat, df=f_array[2] - f_array[1])

In [16]:
hh = get_hh(template, sens_mat=sim.sens_mat, df=f_array[2] - f_array[1])

In [17]:

template = DataResidualArray(template, f_arr=f_array)
data = DataResidualArray(data[0], f_arr=f_array)
analysis = AnalysisContainer(data_res_arr=data, sens_mat=sim.sens_mat)
analysis.template_inner_product(template=template)

44715.71355907657

Inner product changes from 44374.39799894732 to -503215960796.4901 if you remove the line self.freq[0] = self.freq[1] in LISASimulator

In [24]:
print("optimal snr:", np.sqrt(hh), "detected snr:", dh/(np.sqrt(hh)))
print("snrs from lisatools:", analysis.template_snr(template=template))

optimal snr: 15774.721075949961 detected snr: 2.8346436899771854
snrs from lisatools: (np.float64(15774.721075949961), np.float64(2.8346436899762315))


In [21]:
analysis.template_snr(template=template)

(np.float64(15774.721075949961), np.float64(2.8346436899762315))